In [ ]:
import os
import numpy as np
from skimage import io, segmentation
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob

# ─── USER PARAMETERS ────────────────────────────────────────────────────────────
DAPI_DIR = '/content/drive/MyDrive/biotech/Retina_Lab/VGG_next_level/dapi_img_seg'
SCGN_DIR = '/content/drive/MyDrive/biotech/Retina_Lab/VGG_next_level/scgn_img_seg'
OUT_DIR = '/content/drive/MyDrive/biotech/Retina_Lab/VGG_next_level/extracted_cbcs'
IMAGE_EXT = '.png'
SEG_EXT = '_seg.npy'
OVERLAP_THRESHOLD = 0.3  # Minimum IoU threshold for overlap
NAME_TEMPLATE = 'bipolar_{:04d}.png'
OVERLAY_TMPL = '{}_overlay.png'
# ─

## find IOUs of masks

In [ ]:
os.makedirs(OUT_DIR, exist_ok=True)

def calculate_iou(mask1, mask2):
    """
    Calculate Intersection over Union (IoU) between two binary masks.

    Args:
        mask1: First binary mask
        mask2: Second binary mask

    Returns:
        IoU value between 0 and 1
    """
    # Ensure masks are boolean
    mask1_bool = mask1.astype(bool)
    mask2_bool = mask2.astype(bool)

    # Calculate intersection and union
    intersection = np.logical_and(mask1_bool, mask2_bool).sum()
    union = np.logical_or(mask1_bool, mask2_bool).sum()

    # Avoid division by zero
    if union == 0:
        return 0.0

    iou = intersection / union
    return iou

## Overlay masks for visualisation

In [ ]:
def overlay_masks(dapi_mask, scgn_mask):
    """
    Create an overlay of DAPI and SCGN masks for visualization.

    Args:
        dapi_mask: DAPI segmentation mask
        scgn_mask: SCGN segmentation mask

    Returns:
        RGB overlay image
    """
    # Create RGB overlay
    overlay = np.zeros((*dapi_mask.shape, 3), dtype=np.uint8)

    # DAPI in blue channel
    overlay[:, :, 2] = (dapi_mask > 0) * 255

    # SCGN in red channel
    overlay[:, :, 0] = (scgn_mask > 0) * 255

    # Overlap in magenta (red + blue)
    overlap_region = np.logical_and(dapi_mask > 0, scgn_mask > 0)
    overlay[overlap_region] = [255, 0, 255]  # Magenta for overlap

    return overlay

## Locate matching files

In [ ]:
def find_matching_files(dapi_dir, scgn_dir):
    """
    Find matching DAPI and SCGN image files based on sample and slice identifiers.

    Args:
        dapi_dir: Directory containing DAPI images
        scgn_dir: Directory containing SCGN images

    Returns:
        List of matching file pairs
    """
    dapi_files = glob.glob(os.path.join(dapi_dir, f"C1-*{IMAGE_EXT}"))
    scgn_files = glob.glob(os.path.join(scgn_dir, f"C2-*{IMAGE_EXT}"))

    matching_pairs = []

    for dapi_file in dapi_files:
        # Extract sample and slice info from DAPI filename
        dapi_basename = os.path.basename(dapi_file)
        # Expected format: C1-C30000.png -> extract C30000
        sample_slice = dapi_basename.replace('C1-', '').replace(IMAGE_EXT, '')

        # Look for corresponding SCGN file
        scgn_file = os.path.join(scgn_dir, f"C2-{sample_slice}{IMAGE_EXT}")

        if os.path.exists(scgn_file):
            matching_pairs.append((dapi_file, scgn_file))

    return matching_pairs

## get masks arrays from seg files

In [ ]:

def extract_masks_from_seg(seg_data):
    """
    Extract mask arrays from segmentation data structure.

    Args:
        seg_data: Raw segmentation data from .npy file

    Returns:
        Mask array or None if extraction fails
    """
    try:
        # Handle different data structures
        if isinstance(seg_data, np.ndarray) and seg_data.dtype == 'object':
            data = seg_data.item()
        else:
            data = seg_data

        if isinstance(data, dict):
            masks = data.get('masks', None)
            if masks is None:
                print("Warning: 'masks' key not found in segmentation data")
                return None
        else:
            # Assume the data itself is the mask
            masks = data

        return masks

    except Exception as e:
        print(f"Error extracting masks: {e}")
        return None


## Main function

In [ ]:

def extract_bipolar_cells():
    """
    Main function to extract bipolar cells from DAPI and SCGN image pairs.
    """
    print("Starting bipolar cell extraction...")

    # Find matching DAPI and SCGN files
    matching_pairs = find_matching_files(DAPI_DIR, SCGN_DIR)
    print(f"Found {len(matching_pairs)} matching image pairs")

    if not matching_pairs:
        print("No matching pairs found. Please check file naming convention and paths.")
        return

    cell_counter = 0
    total_overlaps = 0

    for dapi_path, scgn_path in tqdm(matching_pairs, desc="Processing image pairs"):
        try:
            # Load images
            dapi_img = io.imread(dapi_path)
            scgn_img = io.imread(scgn_path)

            # Load corresponding segmentation files
            dapi_base = os.path.splitext(dapi_path)[0]
            scgn_base = os.path.splitext(scgn_path)[0]

            dapi_seg_path = dapi_base + SEG_EXT
            scgn_seg_path = scgn_base + SEG_EXT

            if not os.path.exists(dapi_seg_path) or not os.path.exists(scgn_seg_path):
                print(f"Skipping {os.path.basename(dapi_path)}: missing segmentation files")
                continue

            # Load segmentation data
            dapi_seg_raw = np.load(dapi_seg_path, allow_pickle=True)
            scgn_seg_raw = np.load(scgn_seg_path, allow_pickle=True)

            # Extract masks from segmentation data
            dapi_masks = extract_masks_from_seg(dapi_seg_raw)
            scgn_masks = extract_masks_from_seg(scgn_seg_raw)

            if dapi_masks is None or scgn_masks is None:
                print(f"Skipping {os.path.basename(dapi_path)}: could not extract masks")
                continue

            # Create overlay for visualization
            overlay = overlay_masks(dapi_masks, scgn_masks)
            overlay_name = OVERLAY_TMPL.format(os.path.splitext(os.path.basename(dapi_path))[0])
            io.imsave(os.path.join(OUT_DIR, overlay_name), overlay)

            # Find overlapping regions
            dapi_labels = np.unique(dapi_masks)[1:]  # Exclude background (0)
            scgn_labels = np.unique(scgn_masks)[1:]  # Exclude background (0)

            pair_overlaps = 0

            for dapi_label in dapi_labels:
                dapi_region = (dapi_masks == dapi_label)

                for scgn_label in scgn_labels:
                    scgn_region = (scgn_masks == scgn_label)

                    # Calculate IoU between regions
                    iou = calculate_iou(dapi_region, scgn_region)

                    if iou >= OVERLAP_THRESHOLD:
                        # Extract DAPI region
                        cell_counter += 1
                        pair_overlaps += 1

                        # Get bounding box of DAPI region
                        ys, xs = np.where(dapi_region)
                        if len(ys) == 0 or len(xs) == 0:
                            continue

                        y0, x0 = ys.min(), xs.min()
                        y1, x1 = ys.max() + 1, xs.max() + 1

                        # Extract patch from DAPI image
                        patch = dapi_img[y0:y1, x0:x1].copy()
                        mask_crop = dapi_region[y0:y1, x0:x1]

                        # Apply mask to patch
                        if patch.ndim == 3:
                            patch[~mask_crop] = 0
                        else:
                            patch = patch * mask_crop

                        # Save extracted cell
                        out_name = NAME_TEMPLATE.format(cell_counter)
                        Image.fromarray(patch.astype(np.uint8)).save(os.path.join(OUT_DIR, out_name))

                        print(f"  Extracted cell {cell_counter} (IoU: {iou:.3f})")

            total_overlaps += pair_overlaps
            print(f"  Found {pair_overlaps} overlapping cells in {os.path.basename(dapi_path)}")

        except Exception as e:
            print(f"Error processing {os.path.basename(dapi_path)}: {e}")
            continue

    print(f"\nExtraction complete!")
    print(f"Total cells extracted: {cell_counter}")
    print(f"Total overlapping regions found: {total_overlaps}")


## call everything

In [ ]:
 extract_bipolar_cells()

Starting bipolar cell extraction...
Found 18 matching image pairs


Processing image pairs:   0%|          | 0/18 [00:00<?, ?it/s]

  Extracted cell 1 (IoU: 0.598)
  Extracted cell 2 (IoU: 0.709)
  Extracted cell 3 (IoU: 0.756)
  Extracted cell 4 (IoU: 0.677)
  Extracted cell 5 (IoU: 0.764)
  Extracted cell 6 (IoU: 0.381)
  Extracted cell 7 (IoU: 0.581)
  Extracted cell 8 (IoU: 0.402)
  Extracted cell 9 (IoU: 0.748)
  Extracted cell 10 (IoU: 0.755)
  Extracted cell 11 (IoU: 0.732)
  Extracted cell 12 (IoU: 0.806)
  Extracted cell 13 (IoU: 0.644)
  Extracted cell 14 (IoU: 0.873)
  Extracted cell 15 (IoU: 0.703)
  Extracted cell 16 (IoU: 0.372)
  Extracted cell 17 (IoU: 0.383)
  Extracted cell 18 (IoU: 0.692)
  Extracted cell 19 (IoU: 0.684)
  Extracted cell 20 (IoU: 0.878)
  Extracted cell 21 (IoU: 0.869)
  Extracted cell 22 (IoU: 0.666)
  Extracted cell 23 (IoU: 0.849)
  Extracted cell 24 (IoU: 0.826)
  Extracted cell 25 (IoU: 0.663)
  Extracted cell 26 (IoU: 0.829)
  Extracted cell 27 (IoU: 0.529)
  Extracted cell 28 (IoU: 0.854)
  Extracted cell 29 (IoU: 0.599)


Processing image pairs:   6%|▌         | 1/18 [00:29<08:15, 29.15s/it]

  Found 29 overlapping cells in C1-C40001.png
  Extracted cell 30 (IoU: 0.535)
  Extracted cell 31 (IoU: 0.692)
  Extracted cell 32 (IoU: 0.379)
  Extracted cell 33 (IoU: 0.499)
  Extracted cell 34 (IoU: 0.684)
  Extracted cell 35 (IoU: 0.646)
  Extracted cell 36 (IoU: 0.668)
  Extracted cell 37 (IoU: 0.786)
  Extracted cell 38 (IoU: 0.588)
  Extracted cell 39 (IoU: 0.591)
  Extracted cell 40 (IoU: 0.677)
  Extracted cell 41 (IoU: 0.726)
  Extracted cell 42 (IoU: 0.757)
  Extracted cell 43 (IoU: 0.542)
  Extracted cell 44 (IoU: 0.796)
  Extracted cell 45 (IoU: 0.786)
  Extracted cell 46 (IoU: 0.777)
  Extracted cell 47 (IoU: 0.746)
  Extracted cell 48 (IoU: 0.625)
  Extracted cell 49 (IoU: 0.795)
  Extracted cell 50 (IoU: 0.820)
  Extracted cell 51 (IoU: 0.667)


...

  Found 32 overlapping cells in C1-C30003.png
  Extracted cell 628 (IoU: 0.578)
  Extracted cell 629 (IoU: 0.672)
  Extracted cell 630 (IoU: 0.812)
  Extracted cell 631 (IoU: 0.750)
  Extracted cell 632 (IoU: 0.588)
  Extracted cell 633 (IoU: 0.462)
  Extracted cell 634 (IoU: 0.332)
  Extracted cell 635 (IoU: 0.426)
  Extracted cell 636 (IoU: 0.778)
  Extracted cell 637 (IoU: 0.800)
  Extracted cell 638 (IoU: 0.746)
  Extracted cell 639 (IoU: 0.806)
  Extracted cell 640 (IoU: 0.701)
  Extracted cell 641 (IoU: 0.492)
  Extracted cell 642 (IoU: 0.709)


Processing image pairs:  94%|█████████▍| 17/18 [08:48<00:23, 23.83s/it]

  Found 15 overlapping cells in C1-C30004.png
  Extracted cell 643 (IoU: 0.747)


Processing image pairs: 100%|██████████| 18/18 [09:02<00:00, 30.11s/it]

  Found 1 overlapping cells in C1-C30005.png

Extraction complete!
Total cells extracted: 643
Total overlapping regions found: 643
